In [1]:
import pickle
import numpy as np
# import pandas as pd
import os
from openai import OpenAI
import httpx

def save_pickle(obj, filename):
    _, ext = os.path.splitext(filename)
    if ext in ['.pkl','.p','.data']:
        with open(filename, "wb") as f:
            pickle.dump(obj, f)
    elif ext == '.npy':
        if not isinstance(obj, np.ndarray):
            obj = np.array(obj)
        np.save(filename, obj)
    else:
        pass # raise Error

def load_pickle(filename):
    _, ext = os.path.splitext(filename)
    if ext in ['.pkl','.p','.data']:
        with open(filename, "rb") as f:
            data = pickle.load(f)
        return data
    elif ext == '.npy':
        return np.load(filename)
    else:
        return None # raise Error


In [2]:
# read from down sampling results
# apply sudo label assignment

import regex
import json

# down_sampling_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/input_sample.jsonl"
# down_sampling_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/generation_1/generation_1.jsonl"
down_sampling_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1//generation_1.jsonl"

regex_pattern = r"\[\d+\] ([\w\s]+)"

def read_from_topicgpt(filepath):
    user_tweets = {}
    with open(filepath, "r") as f:
        for line in f:
            info = json.loads(line)
            text = info["text"]
            mt_info = info["meta_info"]
            # topic = info["responses"]
            # topic = regex.compile(regex_pattern).findall(topic)
            # if len(topic) > 0: topic = topic[0]
            # else: topic = "None"
            if mt_info["user"] not in user_tweets: user_tweets[mt_info["user"]] = []
            user_tweets[mt_info["user"]].append({
                "user": mt_info["user"],
                "ts": mt_info["ts"],
                "text": text,
                "tag": mt_info["tag"],
                "bertopic_label": mt_info["bertopic_label"],
                # "topicgpt_label": topic,
            })
    return user_tweets

user_tweets = read_from_topicgpt(down_sampling_filepath)
print(len(user_tweets))

2000


In [12]:
user_tweets[4026]

[{'user': 4026,
  'ts': 1584196484,
  'text': 'RT @MirMAKOfficial: #Karachi just passed an order to shut down everything.\n\nMarriage Halls. Malls. Seaside. Gyms. Banquet halls. Jalsas. UR…',
  'tag': '#karachi',
  'bertopic_label': '12'},
 {'user': 4026,
  'ts': 1580914547,
  'text': 'RT @MirMAKOfficial: Listen To This:\n\n#CMSindh gives Rs 42 million of educational funds meant for poor and orphan students to the children o…',
  'tag': '#cmsindh',
  'bertopic_label': '10'},
 {'user': 4026,
  'ts': 1584723344,
  'text': "RT @PAF_Falcons: #SindhGovernment set a field hospital &amp; established quarantine centre at Pakistan's biggest city Karachi's Expo Centre wit…",
  'tag': '#sindhgovernment',
  'bertopic_label': '12'},
 {'user': 4026,
  'ts': 1577547849,
  'text': 'RT @kavita_krishnan: #Thread on visit to #Muzaffarnagar, UP with @harsh_mander @natashabadhwar @JohnDayal @Sandeep_Saurav_ &amp; many others.\nI…',
  'tag': '#muzaffarnagar',
  'bertopic_label': '17'},
 {'user': 4026,
  'ts

In [ ]:
# find unsupervised infos
unsupervised_infos = {}

def is_in_list(item, list, return_elem=False):
    flag = False
    elem = None
    for e in list:
        if e["user"] == item["user"] and \
            e["ts"] == item["ts"] and \
            e["text"] == item["text"] and \
            e["tag"] == item["tag"] and \
            e["bertopic_label"] == item["bertopic_label"]:
            flag = True
            elem = e
        if flag: break
    if not return_elem: return flag
    else: return elem

for user, infos in user_tweets.items():
    full_infos = user_tweets_dict[user]
    # print(len(full_infos))
    unsampled_infos = []
    for info in full_infos:
        if is_in_list(info, infos): continue
        unsampled_infos.append(info)
    unsupervised_infos[user] = unsampled_infos

# analyse_distribution([len(v) for k, v in unsupervised_infos.items()])
print(len(user_tweets))

In [ ]:
# 2. Use trained bertopic model to assign sudo labels

from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_20newsgroups

# Get labeled data
# data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
# docs = data['data']
# y = data['target']


# Skip over dimensionality reduction, replace cluster model with classifier,
# and reduce frequent words while we are at it.
empty_dimensionality_model = BaseDimensionalityReduction()
clf = LogisticRegression()
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

# Create a fully supervised BERTopic instance
topic_model= BERTopic(
    umap_model=empty_dimensionality_model,
    hdbscan_model=clf,
    ctfidf_model=ctfidf_model
)
topics, probs = topic_model.fit_transform(docs, y=y)


In [ ]:
topic_model.get_topic_info()